# Modelización mediante una regresión lineal

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

#### 1. Carga de datos (precios de casas en Boston)

In [2]:
boston = datasets.load_boston()
print(boston.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

#### 2. Observación de los datos

In [4]:
ind = 5
print("---VARIABLES EXPLICATIVAS---")
for vble_name, value in zip(boston['feature_names'], boston['data'][ind]):
    print(vble_name + ": " + str(value))
print("\n---VARIABLE OBJETIVO---")
print("MEDV: " + str(boston['target'][ind]))

---VARIABLES EXPLICATIVAS---
CRIM: 0.02985
ZN: 0.0
INDUS: 2.18
CHAS: 0.0
NOX: 0.458
RM: 6.43
AGE: 58.7
DIS: 6.0622
RAD: 3.0
TAX: 222.0
PTRATIO: 18.7
B: 394.12
LSTAT: 5.21

---VARIABLE OBJETIVO---
MEDV: 28.7


#### 3. Separación en un conjunto de entrenamiento (70%) y otro de test (30%)

In [5]:
X = pd.DataFrame(boston['data'], columns = boston['feature_names'])
y = pd.DataFrame(boston['target'], columns = ['MEDV'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1018)
X_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
491,0.10574,0.0,27.74,0.0,0.6090,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07
211,0.37578,0.0,10.59,1.0,0.4890,5.404,88.6,3.6650,4.0,277.0,18.6,395.24,23.98
350,0.06211,40.0,1.25,0.0,0.4290,6.490,44.4,8.7921,1.0,335.0,19.7,396.90,5.98
380,88.97620,0.0,18.10,0.0,0.6710,6.968,91.9,1.4165,24.0,666.0,20.2,396.90,17.21
331,0.05023,35.0,6.06,0.0,0.4379,5.706,28.4,6.6407,1.0,304.0,16.9,394.02,12.43


#### 4. Instanciación de un modelo de regresión lineal

In [7]:
regr = linear_model.LinearRegression()

#### 5. Entrenamiento (construcción) y guardado del modelo

In [8]:
regr.fit(X_train, y_train)
joblib.dump(regr, 'model.pkl')

print("   COEFICIENTES DE LA REGRESIÓN   ")
print("----------------------------------")
print(regr.coef_)

   COEFICIENTES DE LA REGRESIÓN   
----------------------------------
[[-1.10329111e-01  5.66997361e-02  6.72411687e-02  3.40675302e+00
  -1.87922895e+01  2.97989171e+00 -8.07715162e-03 -1.70215951e+00
   3.01797822e-01 -1.23099758e-02 -9.39754754e-01  8.50202554e-03
  -5.56888218e-01]]


#### 6. Validación (testeo) del modelo

In [9]:
y_pred = regr.predict(X_test)
msq = np.round(mean_squared_error(y_test, y_pred), 2)
r2  = np.round(100 * r2_score(y_test, y_pred), 2)
print("Error cuadrático medio: " + str(msq))
print("Varianza explicada por el modelo: " + str(r2) + "%")

Error cuadrático medio: 20.8
Varianza explicada por el modelo: 77.14%


#### 7. Convertimos los datos de test a JSON

In [8]:
data = X_test.to_json(orient='records')
data[0:1000]

'[{"CRIM":0.10574,"ZN":0.0,"INDUS":27.74,"CHAS":0.0,"NOX":0.609,"RM":5.983,"AGE":98.8,"DIS":1.8681,"RAD":4.0,"TAX":711.0,"PTRATIO":20.1,"B":390.11,"LSTAT":18.07},{"CRIM":0.37578,"ZN":0.0,"INDUS":10.59,"CHAS":1.0,"NOX":0.489,"RM":5.404,"AGE":88.6,"DIS":3.665,"RAD":4.0,"TAX":277.0,"PTRATIO":18.6,"B":395.24,"LSTAT":23.98},{"CRIM":0.06211,"ZN":40.0,"INDUS":1.25,"CHAS":0.0,"NOX":0.429,"RM":6.49,"AGE":44.4,"DIS":8.7921,"RAD":1.0,"TAX":335.0,"PTRATIO":19.7,"B":396.9,"LSTAT":5.98},{"CRIM":88.9762,"ZN":0.0,"INDUS":18.1,"CHAS":0.0,"NOX":0.671,"RM":6.968,"AGE":91.9,"DIS":1.4165,"RAD":24.0,"TAX":666.0,"PTRATIO":20.2,"B":396.9,"LSTAT":17.21},{"CRIM":0.05023,"ZN":35.0,"INDUS":6.06,"CHAS":0.0,"NOX":0.4379,"RM":5.706,"AGE":28.4,"DIS":6.6407,"RAD":1.0,"TAX":304.0,"PTRATIO":16.9,"B":394.02,"LSTAT":12.43},{"CRIM":0.26169,"ZN":0.0,"INDUS":9.9,"CHAS":0.0,"NOX":0.544,"RM":6.023,"AGE":90.4,"DIS":2.834,"RAD":4.0,"TAX":304.0,"PTRATIO":18.4,"B":396.3,"LSTAT":11.72},{"CRIM":1.61282,"ZN":0.0,"INDUS":8.14,"CHAS":0